<a href="https://colab.research.google.com/github/KauaHenSilva/python_optical_flow_sensor/blob/main/optical_flow_dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacotes Externos

In [1]:
import os

In [2]:
if os.path.exists('python_download_file_drive'):
  !cd python_download_file_drive && git pull
else:
  !git clone https://github.com/KauaHenSilva/python_download_file_drive

Cloning into 'python_download_file_drive'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 6.51 KiB | 6.51 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
%pip install  --upgrade --quiet -r python_download_file_drive/requirements.txt

# baixando arquivos

In [4]:
import zipfile

In [5]:
!python "python_download_file_drive/main.py" "https://drive.google.com/file/d/1xQtcYzenMZg7OWd90aBTKf_wBZf5E6Xr/view?usp=drive_link" arq.zip --fuzzy
if not os.path.exists('arq') and os.path.exists('arq.zip'):
    with zipfile.ZipFile('arq.zip', 'r') as zip_ref:
        zip_ref.extractall('arq')

Downloading...
From (original): https://drive.google.com/uc?id=1xQtcYzenMZg7OWd90aBTKf_wBZf5E6Xr
From (redirected): https://drive.google.com/uc?id=1xQtcYzenMZg7OWd90aBTKf_wBZf5E6Xr&confirm=t&uuid=f9573e4b-c1a4-4ae3-8b08-660afdcc4029
To: /content/arq.zip
100% 394M/394M [00:04<00:00, 81.0MB/s]


#  Optical Flow Dense

In [6]:
import cv2
import numpy as np

In [7]:
cap = cv2.VideoCapture("/content/arq/Rastreamento_OpenCV/videos/walking.avi")
ret, frame_init = cap.read()
if not ret:
  raise Exception("Não foi possível ler o vídeo de inicialização.")

In [8]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24
video_saida = cv2.VideoWriter("resultado.avi", fourcc, fps, (frame_init.shape[1], frame_init.shape[0]))

In [9]:
frame_gray_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame_init)
hsv[..., 1] = 255

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(frame_gray_init, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    final = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    video_saida.write(final)
    frame_gray_init = frame_gray.copy()

cap.release()
video_saida.release()

# Exibição de resultado

ATENÇÃO: Use o Colab ou VSCode para visualizar os resultados.

In [12]:
%pip install --upgrade --quiet ffmpeg-python
import ffmpeg
import base64
import io
from IPython.display import HTML
import tempfile

In [13]:
def compress_video(input_file, output_file, resolution=(640, 360), bitrate="500k"):
    ffmpeg.input(input_file).output(output_file, vf=f'scale={resolution[0]}:{resolution[1]}', video_bitrate=bitrate).run(overwrite_output=True,  quiet=True)

def show_video(name_file, otimized=False):
    if otimized:
        with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as temp_video:
            compressed_file = temp_video.name
            compress_video(name_file, compressed_file)
        name_file = compressed_file

    video = io.open(name_file, 'r+b').read()
    encoded = base64.b64encode(video).decode('ascii')

    return HTML(data=f'''
    <video width="640" height="360" controls>
        <source src="data:video/mp4;base64,{encoded}" type="video/mp4">
    </video>''')

In [14]:
!ffmpeg -y -loglevel panic -i resultado.avi resultado.mp4
show_video("resultado.mp4", otimized=True)